<a href="https://colab.research.google.com/github/benihime91/pytorch_retinanet/blob/master/001_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Introduction**

In this notebook, we implement [PyTorch RetinaNet](https://github.com/benihime91) for custom dataset. 

We will take the following steps to implement PyTorch RetinaNet on our custom data:
* Install PyTorch RetinaNet along with required dependencies.
* Download Custom Dataset.
* Write Training Configuation yaml file .
* Train  Detection Model .
* Use Trained TensorFlow2 Object Detection For Inference on Test Images.

When you are done you will have a custom detector that you can use. It will make inference like this:

#### ![Roboflow Workmark](https://i.imgur.com/L0n564N.png)

### **Setting up Colab :**

In [ ]:
# What GPU do we have ?
!nvidia-smi

In [ ]:
# Ensure colab doesn't disconnect
%%javascript
function ClickConnect(){
console.log("Working");
document.querySelector("colab-toolbar-button#connect").click()
}setInterval(ClickConnect,60000)

# **Install Pytorch Retinanet and Dependencies** :

In [ ]:
# Clone the RetinaNet Repo
!git clone https://github.com/benihime91/pytorch_retinanet.git
# install dependencies
!pip install pytorch-lightning omegaconf --quiet
!pip install git+https://github.com/albumentations-team/albumentations --quiet
!echo "[   OK   ] Installed all depedencies "

#Update sys path to enclude the pytorch retinaet modules
import warnings
import os
import sys

warnings.filterwarnings('ignore')
sys.path.append("/content/pytorch_retinanet/")
%load_ext autoreload
%autoreload 2
%matplotlib inline

!echo "[   OK   ] Setup Done "

**Load in Google Drive :**

In [ ]:
# mount google drive
from google.colab import drive
drive.mount("/content/drive")

# **Prepare Pytorch Retinanet Object Detection Training Data**

We will use the **[BCCD Dataset](https://public.roboflow.com/object-detection/bccd)** from RoboFlow.  This is a dataset of blood cells photos.  

There are 364 images across three classes: 
* WBC (white blood cells)
* RBC (red blood cells)
* Platelets. 


There are 4888 labels across 3 classes (and 0 null examples).


To train on the custom dataset the data needs to be in either **csv** or **pascal-voc** format . Roboflow makes it easier to generate the datasets. We can directly download the datsets in required format.

We will download the dataset in **Pascal-VOC** format and then use in-built methods available in PyTorch Retinanet to convert our data into **csv** format.

In [ ]:
#Downloading data from Roboflow
#UPDATE THIS LINK - get our data from Roboflow
%cd /content
#curl -L "[YOUR LINK HERE]" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip
!curl -L "https://public.roboflow.com/ds/vYOavmoHyM?key=byxE8x2t11" > roboflow.zip
!unzip roboflow.zip
!rm roboflow.zip

In [ ]:
#Set up paths 

#Path to where the Images are stored
TRAIN_IMAGES = "/content/train"
VALID_IMAGE = "/content/valid"
TEST_IMAGES = "/content/test"
#Path to where the xml annotations are stored
TRAIN_ANNOTATIONS = "/content/train"
VALID_ANNOTATIONS = "/content/valid"
TEST_ANNOTATIONS  = "/content/test"

In [ ]:
import pandas as pd
from utils.pascal import convert_annotations_to_df
from PIL import Image
import cv2

pd.set_option("display.max_colwidth", None)

## **Generate csv file from XML Annotations:**

In [ ]:
#convert xml files to pandas DataFrames
train_df = convert_annotations_to_df(TRAIN_ANNOTATIONS, TRAIN_IMAGES, image_set="train")
valid_df = convert_annotations_to_df(VALID_ANNOTATIONS, VALID_IMAGE, image_set="test")
test_df  = convert_annotations_to_df(TEST_ANNOTATIONS, TEST_ANNOTATIONS, image_set="test")
!echo "[   DONE  ] DataFrames Generated !"

# removing annotations that are not valid annotations
!echo "[   INFO  ] Removing invalied annotations ... "
train_df = train_df[train_df.xmax > train_df.xmin]
train_df = train_df[train_df.ymax > train_df.ymin]

valid_df = valid_df[valid_df.xmax > valid_df.xmin]
valid_df = valid_df[valid_df.ymax > valid_df.ymin]

test_df = test_df[test_df.xmax > test_df.xmin]
test_df = test_df[test_df.ymax > test_df.ymin]

train_df.reset_index(inplace=True, drop=True)
valid_df.reset_index(inplace=True, drop=True)
test_df.reset_index(inplace=True, drop=True)

#save the csv files to memory
TRAIN_CSV = "/content/train/train_data.csv"
VALID_CSV = "/content/valid/valid_data.csv"
TEST_CSV  = "/content/test/test_data.csv"

train_df.to_csv(TRAIN_CSV, index=False)
!echo "[   DONE  ] Train csv files saved to {TRAIN_CSV} !"

valid_df.to_csv(VALID_CSV, index=False)
!echo "[   DONE  ] Valid csv files saved to {VALID_CSV} !"

test_df.to_csv(TEST_CSV, index=False)
!echo "[   DONE  ] Test csv files saved to {TEST_CSV} !"

### **CSV Files are as follows :**

In [ ]:
train_df.head()

In [ ]:
valid_df.head()

In [ ]:
test_df.head()

# **View Images from the Dataset** :

We can use the fn `visualize_boxes_and_labels_on_image_array` from the RetinaNet repo to visualize images and the bounding boxes over them. To use this function we need to first create a Label Map, which is a list that contains all the classes at index corresponding to the integer labels .

##**Let's now generate the Label Map which is used for visualization:**

In [ ]:
from utils.pascal import generate_pascal_category_names

LABEL_MAP = generate_pascal_category_names(train_df)
LABEL_MAP

##**Plot images with Bounding boxes over them**:

In [ ]:
from utils import visualize_boxes_and_labels_on_image_array
import numpy as np

In [ ]:
def grab_bbs_(dataframe, index:int):
    """
    Takes in a Pandas DataFrame and a index number
    Returns filename of the image and all the bounding boxes and class_labels
    corresponding the image that is at the given index
    """
    assert index <= len(dataframe), f"[  ERROR  ] Invalid index for dataframe with len: {len(dataframe)}"
    fname = train_df.filename[index]
    locs = train_df.loc[train_df.filename == fname]
    bbs  = locs[["xmin", "ymin", "xmax", "ymax"]].values
    cls = locs["labels"].values
    return fname, bbs, cls

### **Image from Train Data:**

In [ ]:
#grab image, boxe and class target
image, boxes, clas = grab_bbs_(train_df, index=0)

#load and normalize the image
image = Image.open(image)
image = np.array(image) / 255.

#draw boxes over the image
image = visualize_boxes_and_labels_on_image_array(
                image=image,
                boxes=boxes, 
                scores=None, 
                classes=clas,
                label_map=LABEL_MAP,
)

image

### **Image from Validation data:**

In [ ]:
#grab image, boxe and class target
image, boxes, clas = grab_bbs_(valid_df, index=50)

#load and normalize the image
image = Image.open(image)
image = np.array(image) / 255.

#draw boxes over the image
image = visualize_boxes_and_labels_on_image_array(
                image=image,
                boxes=boxes, 
                scores=None, 
                classes=clas,
                label_map=LABEL_MAP,
)

image

###**Image from Test Data:**

In [ ]:
#grab image, boxe and class target
image, boxes, clas = grab_bbs_(test_df, index=100)

#load and normalize the image
image = Image.open(image)
image = np.array(image) / 255.

#draw boxes over the image
image = visualize_boxes_and_labels_on_image_array(
                image=image,
                boxes=boxes, 
                scores=None, 
                classes=clas,
                label_map=LABEL_MAP,
)

image

#**Configure Custom PyTorch RetianNet Object Detection Training Configuration** :


In [ ]:
#let's look at the config file
%cat /content/pytorch_retinanet/hparams.yaml

In [ ]:
from omegaconf import OmegaConf

#load in the hparams.ymal file using Omegaconf
hparams = OmegaConf.load("/content/pytorch_retinanet/hparams.yaml")
!echo "[  WORKING ] Writing Custom Configuration File .. "
# ========================================================================= #
# MODIFICATION OF THE CONFIG FILE TO FIX PATHS AND DATSET-ARGUEMENTS :
# ========================================================================= #
#specify kind of data to use
hparams.dataset.kind = "csv"
#Paths to the csv files
hparams.dataset.trn_paths   = TRAIN_CSV
hparams.dataset.valid_paths = VALID_CSV
hparams.dataset.test_paths  = TEST_CSV
# Dataset has 3 classes excluding the "__background__" class
hparams.model.num_classes = 3
#Changing optimizer paramters, 
#Scheduler can also be Optimized in the same way
hparams.optimizer = {
                "class_name": "torch.optim.AdamW", 
                "params": {
                    "lr": 3e-04,
                    "weight_decay" : 1e-03,
                    }
            }

!echo "[   DONE   ] Configuration File : "
print(OmegaConf.to_yaml(hparams))

#**Instantiate Lightning-Module and Lightning-Trainer**

In [ ]:
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import *
from pytorch_lightning.callbacks import *

from model import RetinaNetModel, LogCallback
import time

# seed so that results are reproducible
pl.seed_everything(123)

##**Load in the Lighning-Module using the hparams file modified above :**

In [ ]:
# Instantie lightning-module
litModel = RetinaNetModel(hparams=hparams)

## **Load in the Lighning-Trainer :**

In [ ]:
# ============================================================ #
# INSTANTIATE LIGHTNING-TRAINER with CALLBACKS :
# ============================================================ #
# NOTE: 
# For a list of whole trainer specific arguments see : 
# https://pytorch-lightning.readthedocs.io/en/latest/trainer.html

# Wandb logger
# can use any other logger
save_dir = "/content/logs"
LOGGER = TensorBoardLogger(save_dir="/content/logs")
# Learning-rate Logger
LR_LOGGER = LearningRateLogger(logging_interval="step")

# Model Checkpoint
fname =f"/content/drive/My Drive/demo_rbc/"
os.makedirs(fname, exist_ok=True)
CHECKPOINT_CALLBACK = ModelCheckpoint(fname, mode="min", monitor="val_loss", save_top_k=3,)

# callback for early-stopping
EARLY_STOPPING_CALLBACK = EarlyStopping(mode="min", monitor="val_loss", patience=10,)

# instantiate trainer with arguments from above
trainer = Trainer(precision=16, 
                  max_epochs=50,
                  num_sanity_val_steps=0,
                  gpus=1, 
                  logger=[LOGGER],
                  early_stop_callback=EARLY_STOPPING_CALLBACK, 
                  checkpoint_callback=CHECKPOINT_CALLBACK,
                  callbacks=[LogCallback(), LR_LOGGER], 
                  weights_summary=None,
                  terminate_on_nan=True, 
                  benchmark=True,
                  )

# **Train Custom Pytorch Retinanet Object Detector**:

To train using pytorch-lightning we just need to run `trainer.fit(litModel)`.

Properties (like data, optimzier, scheduler) are already defined in the `litModel` which was loaded using the custom modified `hparams` file.


While `trainer` handles properties like Callbacks, fp_16, GPU ...

In [ ]:
!echo "[   START   ] START TRAINING ... "
trainer.fit(litModel)
!echo "[    END    ] TRAINING COMPLETE ! "
!echo "[    INFO   ] RUN  : %tensorboard --logdir {save_dir} to view TensorBoard Logs"

### **Open TensorBoard Logs :**

In [ ]:
%load_ext tensorboard
%tensorboard --logdir {save_dir}

#**Evaluating the trained-model using COCO-API Metrics** : 

In [ ]:
# Evaluations results on the test/ validation dataset(if test dataset is not given)
# using COCO API
!echo "[   START   ] START EVALUATION OF MODEL ON TEST IMAGE USING COCO-API ... "
trainer.test(litModel)
!echo "[    END    ] DONE EVALUATING MODEL ON TEST IMAGE USING COCO-API ! "

# **Export the model weights** :

In [ ]:
import torch

PATH = f"/content/drive/My Drive/pascal_weights_{int(time.time())}.pth"
torch.save(litModel.model.state_dict(), path)

#**Load PyTroch Model from the trained Lightning-Module weights :**

In [ ]:
from retinanet import Retinanet

state_dict = torch.load(PATH)

MODEL = Retinanet(num_classes=hparams.model.num_classes, backbone_kind=hparams.model.backbone_kind)
MODEL.load_state_dict(state_dict)

# **Run Inference on Test Images with Custom PyTroch Object Detector**

In [ ]:
from PIL import Image
import numpy as np
import cv2

import albumentations as A
from albumentations.pytorch import ToTensorV2

from utils import visualize_boxes_and_labels_on_image_array

@torch.no_grad()
def get_preds(path):
    """
    Generates predictions on the given image from the given path.
    """
    image = cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)
    
    INFER_TRANSFORMS = A.Compose([A.ToFloat(max_value=255.0, always_apply=True),
                                  ToTensorV2(always_apply=True)
                                  ])
    
    TENSOR_IMAGE = INFER_TRANSFORMS(image=image)["image"]
    PREDICTIONS = MODEL.predict([TENSOR_IMAGE])
    return PREDICTIONS[0]

def filter_preds(ps, threshold=0.5):
    """
    Filters the predictions using given threshold.
    """
    scores = ps["scores"]
    labels = ps["labels"]
    boxes = ps["boxes"]

    mask = scores > threshold

    scores = scores[mask]
    labels = labels[mask]
    boxes = boxes[mask]
    return scores.numpy(), labels.numpy(), boxes.numpy()


def detect(image_path, threshold=0.5):
    """
    Generate detections on the image that is present in 
    the given image path

    Args:
        image_path: Path to the input Image
        threshold: Score threshold to filter predictions

    Returns: a PIL image containg the original Image and
             bounding boxes draw over it.
    """
    
    # visualize_boxes_and_labels_on_image_array function
    # expects the pixels values of the image to be in 
    # range [0,1] so be divide the loaded image by 255.0
    # to noramlize the co-ordinates
    image = Image.open(image_path)
    image = np.array(image) / 255.
    
    # Generate predictions for the given image
    preds = get_preds(image_path)
    # Filter predictions
    scores, labels, boxes = filter_preds(preds, threshold)
    # Draw all the bounding boxes over the Image
    im = visualize_boxes_and_labels_on_image_array(
        image,
        boxes,
        labels,
        scores,
        PASCAL_INSTANCE_CATEGORY_NAMES)
    
    return im

In [ ]:
PATH = ...
THRESHOLD = 0.5 #@param {type: "number"}

IMAGE = detect(PATH, THRESHOLD)
IMAGE